In [ ]:
import cv2
import torch
import time
import numpy as np
import serial
from PIL import ImageFont, ImageDraw, Image
import requests # flask에 데이터 전송위한 라이브러리


# 화면에 한글을 표기하기 위한 함수
def myPutText(src, text, pos, font_size, font_color):
    img_pil = Image.fromarray(src)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype('AppleGothic.ttf', font_size)
    draw.text(pos, text, font = font, fill = font_color)
    return np.array(img_pil)  # PIL 이미지를 cv2 이미지로 변환.

# 화면이 터치되면 감지하고 아래 동작을 실행
def mouse_handler(event, x, y, flags, param):
    global mouse_loc
    global state
    if event == cv2.EVENT_LBUTTONDOWN: # 마우스 왼쪽 버튼 누름 or 화면 터치
        if state == 0:
            state = 1 # 다음 상태로 넘어가기
        elif state == 2 or state == 3:
            #ser.write(bytes([0b00000000])) # 원래 처음 위치 회전.
            #while(not ser.readable()): pass # Arduino로부터 완료 신호 수신받을때 까지 기다리기, 아두이노가 없다면 주석처리
            #response = ser.read(1)           # 아두이노가 없다면 주석처리
            state = 0
        elif state == 4:
            state = 0
                
    elif event == cv2.EVENT_LBUTTONUP: # 마우스 왼쪽 버튼 땜
        pass

def rotate_dict_and_output_rotate_num(objects, target):
    # 딕셔너리의 키와 값을 리스트로 변환
    keys = list(objects.keys())
    values = list(objects.values())

    # 원하는 물건이 제일 앞에 오도록.
    idx = keys.index(target)
    keys = keys[idx:] + keys[:idx]
    values = values[idx:] + values[:idx]

    # 회전한 키와 값을 다시 딕셔너리로 변환하여 반환
    # 튜플 리스트를 딕셔너리로 변경 가능. # zip은 튜플로 반환함.
    return dict(zip(keys, values)), idx
    
# Color
WHITE = (255, 255, 255)
BLUE = (255, 0, 0)
GREEN = (0, 255, 0)
YELLOW = (0, 255, 255)
BLACK = (0,0,0)

state = 0

# Flask 애플리케이션의 주소 및 엔드포인트
url = "http://eomsanghan.pythonanywhere.com/update"


# read image
logo1 = cv2.imread('logo1.png')
logo2 = cv2.imread('logo2.png')
logo3 = cv2.imread('logo3.png')

cup = cv2.imread('cup_front.png')
QR_origin = cv2.imread('QR_code.png')
QR = cv2.resize(QR_origin,(500,500), interpolation=cv2.INTER_CUBIC)

h1, w1, _ = logo1.shape
h_cup, w_cup, _ = cup.shape

# Initial Image
init_img = np.zeros((640, 640, 3), dtype = np.uint8)
init_img[:,:] = (255, 255, 255)
init_img = myPutText(init_img, '화면의 아무 곳이나 터치하여 다음 단계로 넘어가세요', (75, 600), 20, BLACK)

img = init_img

#VIDEO
VD_ST = cv2.VideoCapture('starbucks_success.mp4')
VD_ED = cv2.VideoCapture('ediya_success.mp4')
VD_B = cv2.VideoCapture('baek_success.mp4')
VD_CP = cv2.VideoCapture('compose_success.mp4')
VD_M = cv2.VideoCapture('mega_success.mp4')
VD_TS = cv2.VideoCapture('ts_success.mp4')
video_dict = {
    'compose':VD_CP,
    'twosome':VD_TS,
    'mega': VD_M,
    'ediya':VD_ED,
    'starbucks': VD_ST,
    'baek' : VD_B
}


# Window
window_name = 'Frame'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
#cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
counter = 0

# Load the video
# 안되면 0부터 1씩 켜서 하기
cap = cv2.VideoCapture(0)
cap_v = None

# classfication
object_list = []

color_dict = {
    'starbucks' : (0, 255, 0),
    'mega' : (97, 150, 242),
    'twosome' : (97, 150, 242),
    'ediya' : (255, 0, 1),
    'compose' : (97, 150, 242),
    'baek' : (97, 150, 242),
    'cup' : (97, 150, 242),
}

brand_dict = {
    'starbucks' : 0,
    'twosome': 1,
    'ediya': 2,
    'compose': 3,
    'mega': 4,
    'baek' : 5,
}

#cup_num = 0 # 0: starbucks 1: mega 2: twosome 3: ediya 4: compose 5: baek

# For USART Communication, 아두이노가 없다면 주석처리
ser = serial.Serial(port = '/dev/cu.usbmodem1101', baudrate = 9600)

# Load the YOLOv5 model with 'best.pt' weights
# 'best_jinmak.pt'
# 'best_final.pt'
# 'best_optim.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best3.pt')

# Set device
#device = 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device).eval()


while True:
    cv2.setMouseCallback(window_name, mouse_handler) # 마우스 이벤트 발생할 때만 실행
    if state == 0:
        img = myPutText(img, '아래 브랜드에 해당하는 컵을 놔주세요', (50, 20), 30, BLACK)
        img[80:80+h1, 14:14+w1] = logo1
        img[80:80+h1, 14+w1: 14+w1*2] = logo2
        img[80:80+h1, 14+w1*2: 14+w1*3] = logo3
        img = myPutText(img, '아래 사진 처럼 로고가 보이게 컵을 놔주세요', (20, 220), 30, BLACK)
        img[300:420,260:380] = cup
        
    elif state == 1:
        img = myPutText(img, '컵을 확인할 때까지 기다려주세요', (90, 20), 30, BLACK)
        cv2.imshow(window_name, img)
        # time.sleep(0.5)
        start_pos = time.time()
        ret, frame = cap.read()
        if not ret:
            print('영상을 사용할 수 없음')
            break
        
        
        # Perform inference on an image
        results = model(frame)
        # Access the detected objects and their labels
        objects = results.pandas().xyxy[0]
        # Print the detected objects and their labels
        for _, obj in objects.iterrows():
            class_name = obj['name']
            if class_name=='cup': continue
            confidence = obj['confidence']
            if confidence > 0.5:
                if class_name not in object_list:      # 동일한 브랜드 여러번 인식 object_list 넣기 제외.
                    object_list.append(class_name)
                cv2.rectangle(frame, (int(obj['xmin']), int(obj['ymin'])), (int(obj['xmax']), int(obj['ymax'])), color_dict[class_name], 2)
                label = f'{class_name}'
                cv2.putText(frame, label, (int(obj['xmin']), int(obj['ymin']) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color_dict[class_name], 2)
        
            
        if len(object_list) == 1: # 여러개 인식 되는 경우 제외시킬려고, 또한 같은 브랜드 여러개 있을 경우 class_name에 추가 X, 따라서 len==1 유지.
            if(('starbucks' in object_list) or ('mega' in object_list) or ('compose' in object_list) or('ediya' in object_list) or ('baek' in object_list)):
                state = 2
                cv2.putText(frame, label , (100, 500), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                brand_dict,count = rotate_dict_and_output_rotate_num(brand_dict, object_list[0]) # 원하는 게 제일 앞에 오도록 회전.
                # 아두이노에 전송할 때, 문자열 전송하는 것이 일반적.
                ser.write(bytes([0b01000000+count]))  # 해당 라벨에 맞는거 회전 (STEP2) 
                while(not ser.readable()): pass # Arduino로부터 완료 신호 수신받을때 까지 기다리기, 아두이노가 없다면 주석처리
                response = ser.read(1)           # 아두이노가 없다면 주석처리
                ser.write(bytes([0b10000000])) #(STEP1)
                request_data = {'user_input': object_list[0]} # 웹 페이지 보낼 데이터
                response = requests.post(url, data=request_data)
                time.sleep(2)
            else:
                state = 3
        
        else:
                state = 3  # 여러 가지 브랜드가 인식 된 경우  "state = 3"
                        
        object_list = []
        latency = time.time() - start_pos
        print("Latency : {}ms".format(int(1000*latency)))
    
    # 인식 한 브랜드 맞는 NX program 실행 상태
    elif state == 2:
        cap_v = video_dict[class_name]
        ret1, frame1 = cap_v.read()
        if not ret1:
            cap_v.set(cv2.CAP_PROP_POS_FRAMES, 0) # 동영상 프레임 처음부터 리셋. Cap 재생 후 다시 재생하기 위해 리셋시키는 역할.
            cv2.destroyWindow('NX-Assembly Sequence'+str(counter))
            counter += 1
            state=4 # QR 코드 띄우는 상태 이동 함수
            pass
        else:
            resize_frame1 = cv2.resize(frame1,(1200,800),interpolation=cv2.INTER_CUBIC)
            cv2.namedWindow('NX-Assembly Sequence'+str(counter), cv2.WINDOW_NORMAL)
            cv2.setWindowProperty('NX-Assembly Sequence'+str(counter), cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
            cv2.imshow('NX-Assembly Sequence'+str(counter),resize_frame1)
        #ret1, frame1 = cap_v.read()
        #if not ret1:
        #    print('영상 사용 안됨')
        #    pass
        
        #else:
            #resize_frame1 = cv2.resize(frame1,(1200,800),interpolation=cv2.INTER_CUBIC)
            #cv2.imshow('video'+str(counter),resize_frame1)
            
    # 인식을 못한 경우  
    elif state == 3:
        time.sleep(1)
        img = myPutText(img, '잘못된 컵의 정보입니다', (155, 20), 30, BLACK)
        img = myPutText(img, '컵을 다시 확인해 주세요', (150, 70), 30, BLACK)
        #img[120:600, 80:560] = frame[:, 80:560], 안되는 이유 찾기
        img[120:600, 80:560] = frame[120:600, 80:560]
        
    # QR 코드 띄우는 창으로 이동 상태    
    elif state == 4:
        img = myPutText(img, '아래 QR 코드로 환급 받아가세요', (90, 20), 30, BLACK)
        cv2.putText(img, class_name, (90, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
        img[120:600, 80:560] = frame[120:600, 80:560]
        img[120:240, 80:200] = cv2.resize(QR, (120,120))
        cv2.imshow(window_name, img)
        
    cv2.imshow(window_name, img)
    img = init_img
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video object and close the display window
cap.release()
VD_ST.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

ser.close()